# Federated Meta-Learning Lab
This notebook demonstrates how to set up a federated meta-learning framework using images from three subfolders in the main dataset directory.

In [1]:
# Import required libraries
import os
import random
from glob import glob
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [2]:
# Define the dataset path and parameters
DATASET_PATH = "D:\\Artificial Intelligence\\Machine Learning\\PhD - Advanced Machine Learning\\GEI Project\\exp - FewShotLearning\\Dataset_fewshot"  # Main dataset folder with subfolders for classes
IMAGE_SIZE = (224, 224)       # Resize all images to 64x64
NUM_CLASSES = 3             # Three classes
CLIENTS = 3                 # Number of federated clients
EPOCHS = 15
BATCH_SIZE = 10

In [3]:
def load_images_from_folder(folder_path, class_label, image_size):
    """Load images from a specific folder and assign a class label."""
    images = []
    labels = []
    image_files = glob(os.path.join(folder_path, "*"))
    for img_file in image_files:
        if img_file.lower().endswith((".jpg", ".jpeg", ".png")):  # Filter for image files
            try:
                img = Image.open(img_file).convert("RGB").resize(image_size)
                images.append(np.array(img) / 255.0)  # Normalize pixel values
                labels.append(class_label)
            except Exception as e:
                print(f"Error loading image {img_file}: {e}")
    print(f"Loaded {len(images)} images from {folder_path}")
    print(labels[0])
    print(images[0])
    return images, labels

In [4]:
# Iterate through subfolders to load data for three classes
def load_dataset(dataset_path, image_size, num_classes):
    all_images = []
    all_labels = []
    subfolders = sorted(os.listdir(dataset_path))[:num_classes]  # First three classes
    print(subfolders)
    for class_idx, subfolder in enumerate(subfolders):
        folder_path = os.path.join(dataset_path, subfolder)
        images, labels = load_images_from_folder(folder_path, class_idx, image_size)
        all_images.extend(images)
        all_labels.extend(labels)
    return np.array(all_images), np.array(all_labels)

In [5]:
# Load the dataset
print("Loading dataset...")
images, labels = load_dataset(DATASET_PATH, IMAGE_SIZE, NUM_CLASSES)
labels = to_categorical(labels, num_classes=NUM_CLASSES)

Loading dataset...
['boxing_GEI', 'handwaving_GEI', 'walking_GEI']
Loaded 100 images from D:\Artificial Intelligence\Machine Learning\PhD - Advanced Machine Learning\GEI Project\exp - FewShotLearning\Dataset_fewshot\boxing_GEI
0
[[[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 ...

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]]
Loaded 100 images from D:\Artificial Intelligence\Machine Learning\PhD - Advanced Machine Learning\GEI Project\exp - FewShotLearning\Dataset_fewshot\handwaving_GEI
1
[[[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]

In [6]:
# Split the data into federated clients
def split_federated_clients(images, labels, num_clients):
    """Distribute the dataset among federated clients."""
    clients_data = []
    data_size = len(images)
    indices = np.arange(data_size)
    random.shuffle(indices)
    split_size = data_size // num_clients
    for i in range(num_clients):
        start_idx = i * split_size
        end_idx = start_idx + split_size if i != num_clients - 1 else data_size
        client_images = images[indices[start_idx:end_idx]]
        client_labels = labels[indices[start_idx:end_idx]]
        clients_data.append((client_images, client_labels))
    return clients_data

clients_data = split_federated_clients(images, labels, CLIENTS)
print(f"Data successfully split among {CLIENTS} clients.")

Data successfully split among 3 clients.


In [7]:
def create_model(input_shape, num_classes):
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [8]:
def train_client(client_data, model, epochs, batch_size):
    """Train the model on client data."""
    client_images, client_labels = client_data
    # Convert images and labels to numpy arrays if needed
    client_images = np.array(client_images)
    client_labels = np.array(client_labels)
    # Check shapes
    # print(f"Client images shape: {client_images.shape}")
    # print(f"Client labels shape: {client_labels.shape}")
    # Train the model
    model.fit(client_images, client_labels, epochs=epochs, batch_size=batch_size, verbose=1)


In [9]:
# Meta-Learning simulation: Iterate through clients and update a global model
global_model = create_model((*IMAGE_SIZE, 3), NUM_CLASSES)
global_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

for round_num in range(1, EPOCHS + 1):
    print(f"Federated Round {round_num}")
    local_models = []
    for client_idx, client_data in enumerate(clients_data):
        print(f"Training on Client {client_idx + 1}")
        # Clone and set the weights of the global model
        local_model = tf.keras.models.clone_model(global_model)
        local_model.set_weights(global_model.get_weights())
        # Compile the local model
        local_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        # Train the local model
        train_client(client_data, local_model, epochs=1, batch_size=BATCH_SIZE)
        # Collect the weights from the local model
        local_models.append(local_model.get_weights())
    
    # Debug: Print the shapes of the collected weights
    # for i, weights in enumerate(local_models):
    #     print(f"Client {i + 1} model weights shapes:")
    #     for weight in weights:
    #         print(weight.shape)
    
    # Aggregate weights (simple average)
    new_weights = []
    for weights_tuple in zip(*local_models):
        new_weights.append(np.mean(weights_tuple, axis=0))
    global_model.set_weights(new_weights)
    global_model.set_weights(new_weights)

c:\Users\Cloud Tech\anaconda3\envs\py310\lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Federated Round 1
Training on Client 1
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 366ms/step - accuracy: 0.3237 - loss: 34.5283
Training on Client 2
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 365ms/step - accuracy: 0.3070 - loss: 26.6451
Training on Client 3
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 364ms/step - accuracy: 0.2693 - loss: 43.9815
Federated Round 2
Training on Client 1
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 366ms/step - accuracy: 0.3681 - loss: 13.0517
Training on Client 2
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 375ms/step - accuracy: 0.2142 - loss: 18.8404
Training on Client 3
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 365ms/step - accuracy: 0.3512 - loss: 14.3372
Federated Round 3
Training on Client 1
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 366ms/step - accuracy: 0.3383 - loss: 1.8595
Training on Client 2
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 369ms/step - accuracy: 0.5046 - loss: 1.4639
Training on Client 3
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 372ms/step - accuracy: 0.4379 - loss: 1.2854
Federated Round 4
Training on Client 1
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 366ms/s

In [10]:
# Evaluate the global model
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2)
test_loss, test_accuracy = global_model.evaluate(test_images, test_labels, verbose=1)
print(f"Global Model Accuracy: {test_accuracy * 100:.2f}%")

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - accuracy: 0.9778 - loss: 0.0316
Global Model Accuracy: 96.67%
